# Исследование базы данных StackOverflow

## Данные

Самостоятельная работа основана на базе данных StackOverflow — сервиса вопросов и ответов о программировании.  
Мы будем работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах есть информация и о более поздних оценках, которые эти посты получили.

## Цель

Проект состоит из двух частей:  
В первой части несколько задач в SQL-тренажёре, чтобы закрепить пройденный материал.  
Вторая часть проекта — аналитическая.

## Задания Часть 1

## Задание 1

Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
'''
SELECT COUNT(pt.id)
FROM stackoverflow.posts p          
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
WHERE (p.score>300 OR p.favorites_count>=100) AND pt.type='Question';
'''

## Задание 2

Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
'''
SELECT ROUND(AVG (count_pt_id),0)
FROM 
(SELECT COUNT(pt.id) AS count_pt_id,
       CAST(p.creation_date AS date)
FROM stackoverflow.posts p          
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
WHERE pt.type='Question'
GROUP BY CAST(p.creation_date AS date)) t
WHERE CAST(creation_date AS date) BETWEEN '2008.11.01' AND '2008.11.18'
''' 

## Задание 3

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
'''
SELECT COUNT(DISTINCT u.id)
FROM StackOverflow.users u
JOIN StackOverflow.badges b ON u.id = b.user_id
WHERE CAST(u.creation_date AS date) = CAST(b.creation_date AS date)
'''

## Задание 4

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
'''
SELECT COUNT(DISTINCT p.id)       
FROM StackOverflow.posts p
JOIN StackOverflow.votes v ON p.id = v.post_id
JOIN StackOverflow.users u ON p.user_id = u.id
WHERE u.display_name ='Joel Coehoorn';
'''

## Задание 5

Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
'''
SELECT *,
       ROW_NUMBER () OVER (ORDER BY id DESC) AS rank
FROM StackOverflow.vote_types
ORDER BY id
'''

## Задание 6

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
'''
SELECT --*
       u.id,
       COUNT (vt.id) 
FROM StackOverflow.users u
JOIN StackOverflow.votes v ON u.id = v.user_id
JOIN StackOverflow.vote_types vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY u.id
ORDER BY COUNT(vt.id) DESC
LIMIT 10;
'''

## Задание 7

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно. Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
'''
SELECT user_id,
       COUNT(id) AS CNT,
       DENSE_RANK() OVER (ORDER BY COUNT(id)DESC)
FROM StackOverflow.badges
WHERE CAST(creation_date AS date) BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY CNT DESC,user_id
LIMIT 10;
'''

## Задание 8

Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
'''
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM stackoverflow.posts
WHERE title IS NOT NULL
AND score != 0
'''

## Задание 9

Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
'''
WITH list_users AS
(
SELECT DISTINCT b.user_id
FROM stackoverflow.badges as b
GROUP BY 1
HAVING COUNT(b.id) > 1000
)
SELECT p.title
FROM list_users as u
JOIN stackoverflow.posts as p ON p.user_id=u.user_id
WHERE p.title IS NOT NULL;
'''

## Задание 10

Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
'''
SELECT id,
       views,
       CASE 
         WHEN views >= 350 THEN 1
         WHEN views < 350 AND views >= 100 THEN 2
         WHEN views < 100 THEN 3
       END  
FROM stackoverflow.users AS u 
WHERE location LIKE '%United States%'
AND views != 0
'''

## Задание 11

Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
'''
WITH b AS 
(SELECT id,
       views,
       CASE 
         WHEN views >= 350 THEN 1
         WHEN views < 350 AND views >= 100 THEN 2
         WHEN views < 100 THEN 3
       END AS groupp 
FROM stackoverflow.users AS u 
WHERE location LIKE '%United States%'
AND views != 0)
SELECT id,
       groupp, 
       views
FROM
(SELECT id,
       MAX(views) OVER (PARTITION BY groupp) as max_views,
       views,
       groupp
FROM b) t
where max_views = views
ORDER BY views DESC, id asc
'''

## Задание 12

Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
номер дня;
число пользователей, зарегистрированных в этот день;
сумму пользователей с накоплением.

In [ ]:
'''
SELECT 
       DISTINCT EXTRACT(DAY FROM creation_date) AS number_day,
       COUNT(id) OVER (PARTITION BY CAST(DATE_TRUNC('day',creation_date) AS date)) as cnt_user,
       COUNT(id) OVER (ORDER BY CAST(DATE_TRUNC('day', creation_date) AS date)) AS cnt_user2
FROM stackoverflow.users
WHERE CAST(creation_date AS date) BETWEEN '2008-11-01' AND '2008-11-30'
'''

## Задание 13

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.

In [ ]:
'''
WITH users AS
(
SELECT DISTINCT p.user_id,
       MIN(creation_date) OVER(PARTITION BY user_id) AS min_d
       --row_number() over (order by MIN(p.creation_date)) as dt
FROM stackoverflow.posts as p
--GROUP BY p.user_id,p.creation_date
)
SELECT u.id,
       us.min_d-creation_date
FROM users as us
JOIN stackoverflow.users as u ON us.user_id=u.id
'''

## Задания Часть 2

## Задание 1

Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
'''
SELECT DATE_TRUNC('month', creation_date)::date AS month,
       SUM(views_count) AS total
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date::date)=2008
GROUP BY month
ORDER BY total DESC
'''

## Задание 2


Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
'''
SELECT --*
       u.display_name,
       COUNT(DISTINCT u.id)
FROM StackOverflow.users u
JOIN StackOverflow.posts p ON u.id = p.user_id
JOIN StackOverflow.post_types pt ON p.post_type_id = pt.id
WHERE pt.type='Answer'
AND (DATE_TRUNC('day', p.creation_date) BETWEEN DATE_TRUNC('day', u.creation_date) AND DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month')
GROUP BY display_name
HAVING COUNT(p.id) > 100
'''

## Задание 3

Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
'''
WITH users AS
(SELECT DISTINCT (u.id)
FROM StackOverflow.users u
JOIN StackOverflow.posts p ON p.user_id=u.id
WHERE u.creation_date::date BETWEEN '2008-09-01' AND '2008-09-30'
AND p.creation_date::date BETWEEN '2008-12-01' AND '2008-12-31')
SELECT DATE_TRUNC('month', p.creation_date)::date AS month,
       COUNT(p.id) AS count
FROM users AS us
JOIN stackoverflow.posts AS p ON us.id=p.user_id
WHERE p.creation_date::date BETWEEN '2008-01-01' AND '2008-12-31'
GROUP BY month
ORDER BY month DESC;
'''

## Задание 4

Используя данные о постах, выведите несколько полей:
идентификатор пользователя, который написал пост;
дата создания поста;
количество просмотров у текущего поста;
сумму просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
'''
SELECT user_id,
       creation_date,
       views_count,
       SUM (views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM StackOverflow.posts
'''

## Задание 5

Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
'''
WITH users AS
(SELECT user_id,
        creation_date::date,
        COUNT(creation_date)AS cd
FROM stackoverflow.posts
WHERE creation_date:: date BETWEEN '2008.12.01' AND '2008.12.07'
GROUP BY user_id,creation_date::date
)
SELECT ROUND(AVG(cd))
FROM users
'''

## Задание 6

На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
номер месяца;
количество постов за месяц;
процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
'''
WITH posts AS       
(SELECT EXTRACT (MONTH FROM creation_date::date) AS month,
       COUNT(id) AS count
FROM stackoverflow.posts
WHERE creation_date:: date BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY month)
SELECT month,
       count,
       ROUND(((count::numeric / LAG(count) OVER (ORDER BY month)) - 1) * 100, 2) AS difference
FROM posts; 
'''

## Задание 7

Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
'''
WITH users AS
(SELECT user_id,
        COUNT(id) AS count
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY count DESC
LIMIT 1)
 
SELECT
    DISTINCT week,
    LAST_VALUE(creation_date) OVER (PARTITION BY EXTRACT(WEEK FROM creation_date))
FROM 
(SELECT EXTRACT(WEEK FROM creation_date) AS week,
        creation_date
FROM stackoverflow.posts p
JOIN users u ON p.user_id = u.user_id
WHERE creation_date::DATE BETWEEN '2008-10-01' AND '2008-10-31'
ORDER BY 1,2) AS ps
'''